# Import libraries

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd
from datetime import datetime

import pyspark
from pyspark.sql.functions import when, col
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, FloatType, TimestampType
from pyspark.sql.functions import avg, datediff, to_date, when, col
from pyspark.sql import functions as F
from pyspark.sql.functions import col, min, max, lag, datediff, lit, coalesce, to_date, unix_timestamp, from_unixtime

# Setup SparkSession

In [22]:
spark = SparkSession.builder \
    .appName("MaintenanceTimerSystem") \
    .getOrCreate()

# Load 2 dataset

In [23]:
df_1 = spark.read.csv("/home/beboy/Desktop/projects/Predictive-Maintenance-System-using-Apache-Spark/dataset/maintenance_logs.csv",header=True, inferSchema = True)
df_1.show(5, truncate=False)

+------------+--------------------------+----------------+--------------------------------+-------------+--------------+---------+--------------+------------------+
|equipment_id|date                      |maintenance_type|description                     |technician_id|duration_hours|cost     |parts_replaced|maintenance_result|
+------------+--------------------------+----------------+--------------------------------+-------------+--------------+---------+--------------+------------------+
|1           |2021-12-22 21:45:39.156781|Repair          |Repair maintenance performed    |44           |4.0433426     |422.3207 |None          |Successful        |
|1           |2022-05-02 21:45:39.156781|Inspection      |Inspection maintenance performed|36           |5.6460757     |4588.3525|None          |Successful        |
|1           |2022-08-15 21:45:39.156781|Inspection      |Inspection maintenance performed|27           |3.9407096     |195.60114|Filters       |Successful        |
|1        

In [24]:
df_2 = spark.read.csv("/home/beboy/Desktop/projects/Predictive-Maintenance-System-using-Apache-Spark/dataset/df_maintenance.csv",header=True, inferSchema = True)
df_2.show(5, truncate=False)

+------------+-----------------------+-----------+----------+----------+----------------+------------+-----------+---------+---------+-------------+----------+-------------+-----------------------+---------------+------------+--------------------+-----------------------+---------------------+-----------------------+---------+-----------+----------------+---------------------------------+-------------+--------------+---------+--------------+------------------+-----------------------+---------------+------------------+------------------+-----------+------------+--------------------+-------------------+----------------+-----------------------+----------------------+------------------+-------------------+------------------+-------------------+------------------+---------------------------+--------------------------+-------------+-------------------+------------------------+--------------------------+--------------------+----------------------------+----------------------+-----------------+

In [25]:
# Lọc dữ liệu với điều kiện maintenance_needed == "No maintenance required"
filtered_df = df_2.filter(col('maintenance_needed') == "No maintenance required")
filtered_df.show(5, truncate=False)

+------------+-----------------------+-----------+----------+----------+----------------+------------+-----------+---------+---------+-------------+----------+-------------+-----------------------+---------------+------------+--------------------+-----------------------+---------------------+-----------------------+---------+-----------+----------------+---------------------------------+-------------+--------------+---------+--------------+------------------+-----------------------+---------------+------------------+------------------+-----------+------------+--------------------+-------------------+----------------+-----------------------+----------------------+------------------+-------------------+------------------+-------------------+------------------+---------------------------+--------------------------+-------------+-------------------+------------------------+--------------------------+--------------------+----------------------------+----------------------+-----------------+

# NOTE QUAN TRỌNG

### Ở phương pháp 1 và 2 có dùng thêm fix_value, vì vẫn có 1 vài id có tổng số giờ vận hành > ngưỡng 
### => 1 vài giá trị âm cần đc + thêm fix_value(đơn vị + vào là hours), chỉ cộng những id có ngưỡng bé hơn tổng giờ vận hành
### khá nhiều id có dao động tổng giờ vận hành hơn 10k giờ, nên fix_value = 10000
### remaining_days giữa các thiết bị bth và thiết bị đc cộng thêm 10k giờ ko chênh lệnh quá nhiều (khoảng dao động hơn 10 ngày)

# Method 1 Usage Based

- Formula:
    1. days_between_maintenance = date - prev_maintenance_date
    2. operational_threshold = avg(days_between_maintenance)
    3. total_current_operating_hours(tổng số giờ vận hành hiện tại) = current_date hiện tại - ngày bảo dưỡng gần nhất(date)
    4. remaining_hours(số ngày còn lại) = operational_threshold(ngưỡng hoạt động) - total_current_operating_hours(tổng số giờ vận hành hiện tại)
    5. remaining_days(số ngày còn lại) = remaining_hours(số giờ còn lại) / avg_daily_operating_hours(số giờ vận hành trung bình mỗi ngày)

In [26]:
def remaining_hours(df):
    # Đặt cửa sổ để tìm ngày bảo dưỡng trước đó cho mỗi thiết bị
    window_spec = Window.partitionBy('equipment_id').orderBy('date')

    # Tính toán 'prev_maintenance_date' và 'days_between_maintenance'
    result_1 = df.withColumn(
        'prev_maintenance_date', F.lag('date').over(window_spec)  # Lấy ngày bảo trì trước đó
    ).withColumn(
        'days_between_maintenance',
        F.datediff(F.col('date'), F.col('prev_maintenance_date'))  # Tính số ngày giữa 2 lần bảo trì
    )
    
    # Tính 'operational_threshold' và join với DataFrame gốc
    avg_threshold = result_1.groupBy('equipment_id') \
        .agg((F.avg('days_between_maintenance') * 24).alias('operational_threshold'))

    result_2 = result_1.join(
        avg_threshold,
        on='equipment_id',
        how='left'
    )
    
    # Đảm bảo rằng cột 'date' là kiểu timestamp
    result_2 = result_2.withColumn('date', F.col('date').cast('timestamp'))

    # Đặt cửa sổ để sắp xếp theo ngày giảm dần
    window_spec = Window.partitionBy('equipment_id').orderBy(F.col('date').desc())

    # Lấy giá trị 'date' cuối cùng cho mỗi 'equipment_id' (ngày bảo dưỡng gần nhất)
    result_2 = result_2.withColumn('last_maintenance_date', F.first('date', ignorenulls=True).over(window_spec))

    # Tính toán tổng số giờ vận hành hiện tại (current operating hours)
    result_2 = result_2.withColumn(
        'total_current_operating_hours', 
        F.datediff(F.current_date(), F.col('last_maintenance_date')) * 24
    )

    # Sử dụng điều kiện để điều chỉnh 'operational_threshold' nếu nó nhỏ hơn 'total_current_operating_hours'
    result_2 = result_2.withColumn(
        'operational_threshold',
        F.when(F.col('operational_threshold') < F.col('total_current_operating_hours'), 
               F.col('operational_threshold') + 10000)
         .otherwise(F.col('operational_threshold'))
    )

    # Tính toán 'remaining_hours' = 'operational_threshold' - 'total_current_operating_hours'
    result_2 = result_2.withColumn(
        'remaining_hours', 
        F.col('operational_threshold') - F.col('total_current_operating_hours')
    )
    
    # Trả về kết quả với cột 'remaining_hours'
    return result_2

# Gọi hàm remaining_hours với DataFrame df hiện tại
result_df1 = remaining_hours(df_1)

# Hiển thị kết quả với thiết bị id 9
result_df1.filter(col('equipment_id') == 9).show(5, truncate=False)

+------------+--------------------------+----------------+--------------------------------+-------------+--------------+---------+--------------+------------------+--------------------------+------------------------+---------------------+--------------------------+-----------------------------+---------------+
|equipment_id|date                      |maintenance_type|description                     |technician_id|duration_hours|cost     |parts_replaced|maintenance_result|prev_maintenance_date     |days_between_maintenance|operational_threshold|last_maintenance_date     |total_current_operating_hours|remaining_hours|
+------------+--------------------------+----------------+--------------------------------+-------------+--------------+---------+--------------+------------------+--------------------------+------------------------+---------------------+--------------------------+-----------------------------+---------------+
|9           |2023-08-15 21:45:39.156781|Inspection      |Inspec

# Avg_daily_operating_hours from df_2

In [27]:
# Loại bỏ các dòng trùng lặp trong df_2 theo equipment_id
filtered_df_unique = filtered_df.dropDuplicates(['equipment_id'])

# Gộp df_2 vào result_df1
new_df1 = result_df1.join(
    filtered_df_unique.select('equipment_id', 'operating_hours'),
    on='equipment_id',
    how='left'
)

# Hiển thị kết quả
new_df1.show(70)

+------------+--------------------+----------------+--------------------+-------------+--------------+----------+--------------+------------------+---------------------+------------------------+---------------------+---------------------+-----------------------------+------------------+------------------+
|equipment_id|                date|maintenance_type|         description|technician_id|duration_hours|      cost|parts_replaced|maintenance_result|prev_maintenance_date|days_between_maintenance|operational_threshold|last_maintenance_date|total_current_operating_hours|   remaining_hours|   operating_hours|
+------------+--------------------+----------------+--------------------+-------------+--------------+----------+--------------+------------------+---------------------+------------------------+---------------------+---------------------+-----------------------------+------------------+------------------+
|           1|2024-11-06 21:45:...|          Repair|Repair maintenanc...|      

In [28]:
# calculate new column avg_daily_operating_hours using F.avg() window function
window_spec = Window.partitionBy('equipment_id').orderBy('date')
new_df1 = new_df1.withColumn(
    'avg_daily_operating_hours', F.avg('operating_hours').over(window_spec)
)

# Hiển thị kết quả
new_df1.show(20)

+------------+--------------------+----------------+--------------------+-------------+--------------+----------+--------------+------------------+---------------------+------------------------+---------------------+---------------------+-----------------------------+---------------+------------------+-------------------------+
|equipment_id|                date|maintenance_type|         description|technician_id|duration_hours|      cost|parts_replaced|maintenance_result|prev_maintenance_date|days_between_maintenance|operational_threshold|last_maintenance_date|total_current_operating_hours|remaining_hours|   operating_hours|avg_daily_operating_hours|
+------------+--------------------+----------------+--------------------+-------------+--------------+----------+--------------+------------------+---------------------+------------------------+---------------------+---------------------+-----------------------------+---------------+------------------+-------------------------+
|         

# Remaining days method 1

In [29]:
# calculate new column remaining_days_method1
# remaining_days_method1 = remaining_hours / avg_daily_operating_hours
final_result1 = new_df1.withColumn(
    'remaining_days_method1', F.round(F.col('remaining_hours') / F.col('avg_daily_operating_hours') / 24, 1)
)

# Hiển thị kết quả
final_result1.show(70)

+------------+--------------------+----------------+--------------------+-------------+--------------+----------+--------------+------------------+---------------------+------------------------+---------------------+---------------------+-----------------------------+------------------+------------------+-------------------------+----------------------+
|equipment_id|                date|maintenance_type|         description|technician_id|duration_hours|      cost|parts_replaced|maintenance_result|prev_maintenance_date|days_between_maintenance|operational_threshold|last_maintenance_date|total_current_operating_hours|   remaining_hours|   operating_hours|avg_daily_operating_hours|remaining_days_method1|
+------------+--------------------+----------------+--------------------+-------------+--------------+----------+--------------+------------------+---------------------+------------------------+---------------------+---------------------+-----------------------------+------------------+-

# Checking

In [30]:
# in ra cột equipment_id, remaining_days_method1 của final_result1 nhưng chỉ in ra 1 giá trị trong mỗi nhóm equipment_id
final_result1.select('equipment_id', 'remaining_days_method1').distinct().show(1000)

+------------+----------------------+
|equipment_id|remaining_days_method1|
+------------+----------------------+
|           1|                   4.5|
|           2|                  15.5|
|           3|                   2.5|
|           4|                  17.5|
|           5|                   3.3|
|           6|                   3.8|
|           7|                   2.3|
|           8|                   0.1|
|           9|                   0.2|
|          10|                  18.3|
|          11|                  17.1|
|          12|                   9.4|
|          13|                   3.4|
|          14|                   3.7|
|          15|                   0.5|
|          16|                   2.7|
|          17|                   1.5|
|          18|                   3.0|
|          19|                  17.3|
|          20|                   3.9|
|          21|                  17.9|
|          22|                  15.3|
|          23|                  17.9|
|          2

# Method 2: Based on maintenance history

Phương pháp 2: Phân tích nhật ký bảo trì

Phương pháp này là một phần của Bước 1 vì nó tập trung vào việc phân tích khoảng thời gian giữa các lần bảo trì và tổng số giờ vận hành trong khoảng đó. Từ đó, ngưỡng hoạt động trung bình của thiết bị được xác định, giúp dự đoán thời gian còn lại trước khi cần bảo trì tiếp theo. Dữ liệu này được tái sử dụng từ việc tính toán trong Bước 1.

In [31]:
# Tính toán remaining_days_method2
final_result2 = final_result1.withColumn(
    'remaining_days_method2', 
    F.round((F.col('operational_threshold') - F.col('total_current_operating_hours')) / 24, 1)  # Làm tròn đến 1 chữ số thập phân
)

# Hiển thị kết quả
final_result2.show(70)

+------------+--------------------+----------------+--------------------+-------------+--------------+----------+--------------+------------------+---------------------+------------------------+---------------------+---------------------+-----------------------------+------------------+------------------+-------------------------+----------------------+----------------------+
|equipment_id|                date|maintenance_type|         description|technician_id|duration_hours|      cost|parts_replaced|maintenance_result|prev_maintenance_date|days_between_maintenance|operational_threshold|last_maintenance_date|total_current_operating_hours|   remaining_hours|   operating_hours|avg_daily_operating_hours|remaining_days_method1|remaining_days_method2|
+------------+--------------------+----------------+--------------------+-------------+--------------+----------+--------------+------------------+---------------------+------------------------+---------------------+---------------------+----

In [32]:
# in ra cột equipment_id, remaining_days_method1 của final_result1 nhưng chỉ in ra 1 giá trị trong mỗi nhóm equipment_id
final_result2.select('equipment_id', 'remaining_days_method1', 'remaining_days_method2').distinct().show(1000)

+------------+----------------------+----------------------+
|equipment_id|remaining_days_method1|remaining_days_method2|
+------------+----------------------+----------------------+
|           1|                   4.5|                  90.0|
|           2|                  15.5|                 341.8|
|           3|                   2.5|                  55.5|
|           4|                  17.5|                 385.2|
|           5|                   3.3|                  66.9|
|           6|                   3.8|                  85.4|
|           7|                   2.3|                  49.6|
|           8|                   0.1|                   3.1|
|           9|                   0.2|                   5.3|
|          10|                  18.3|                 403.4|
|          11|                  17.1|                 376.4|
|          12|                   9.4|                 195.7|
|          13|                   3.4|                  75.8|
|          14|          

# Method 3: Health Score

* Ở đây t không tính theo công thức Số ngày còn lại = (Điểm số hiện tại - Điểm số ngưỡng) / Tốc độ suy giảm điểm số mỗi ngày
* Vì tốc độ suy giảm là điểm giữa 2 lần đo trước và sau trừ cho nhau (điểm 2 lần được đo mỗi khi bảo dưỡng), cột maintenance_date bên df_maintenance đang sai nên t tìm cách khác bên dưới

- vì df_maintenance.csv không có cột date còn maintenance_logs.csv ko có các alert columns
1. giải pháp ở đây là chỉnh lại cột maintenance_date trong df_maintenance.csv lại cho dúng với date trong maintenance_logs.csv

2. gộp cột date trong maintenance_logs.csv vào df_maintenance.csv
    * cách này không ổn vì tuy vẫn 1000 id nhưng df_maintenance.csv mỗi id 4tr dòng, nếu gộp date vào thì khoảng cách giữa các lần bảo dưỡng trùng lặp 

In [33]:
from functools import reduce

def calculate_device_health(df):
    """
    Tính toán các chỉ số sức khỏe thiết bị và thêm vào DataFrame gốc.
    
    Parameters:
    df (spark.DataFrame): DataFrame chứa các cột alert
    
    Returns:
    spark.DataFrame: DataFrame gốc với thêm các cột mới
    """
    # Các tham số cố định
    threshold_score = 60
    base_days = 180  # 6 tháng
    danger_reduction = 0.5  # Giảm 50% 
    warning_reduction = 0.25  # Giảm 25%
    
    # Danh sách các cột alert
    alert_columns = [
        'temperature_alert', 'pressure_alert', 'rotational_speed_alert',
        'power_output_alert', 'noise_level_alert', 'voltage_alert',
        'current_alert', 'oil_viscosity_alert'
    ]
    
    # Tạo DataFrame với các cột deduction
    result_df = df
    
    # Tính điểm trừ cho từng cảnh báo
    for col in alert_columns:
        result_df = result_df.withColumn(
            f'deduction_{col}',
            F.when(F.col(col) == 'Danger', -45)
            .when(F.col(col) == 'Warning', -40)
            .otherwise(0)
        )
    
    # Tính tổng điểm trừ sử dụng functools.reduce
    deduction_cols = [F.col(f'deduction_{col}') for col in alert_columns]
    total_deduction = reduce(lambda x, y: x + y, deduction_cols)
    
    # Tính điểm số hiện tại
    result_df = result_df.withColumn(
        'current_score',
        F.greatest(F.lit(100) + total_deduction, F.lit(0))
    )
    
    # Đếm số lượng cảnh báo Danger
    danger_conditions = [F.when(F.col(col) == 'Danger', 1).otherwise(0) for col in alert_columns]
    danger_sum = reduce(lambda x, y: x + y, danger_conditions)
    
    # Đếm số lượng cảnh báo Warning
    warning_conditions = [F.when(F.col(col) == 'Warning', 1).otherwise(0) for col in alert_columns]
    warning_sum = reduce(lambda x, y: x + y, warning_conditions)
    
    # Thêm các cột phân tích
    result_df = result_df.withColumn(
        'num_danger_alerts',
        danger_sum
    ).withColumn(
        'num_warning_alerts',
        warning_sum
    ).withColumn(
        'health_status',
        F.when(danger_sum > 0, 'Critical')
        .when(warning_sum > 0, 'Warning')
        .otherwise('Healthy')
    ).withColumn(
        'remaining_days_method3',
        F.when(
            F.col('current_score') <= threshold_score,
            0
        ).otherwise(
            F.round(
                F.lit(base_days) * 
                F.pow(F.lit(1 - danger_reduction), F.col('num_danger_alerts')) * 
                F.pow(F.lit(1 - warning_reduction), F.col('num_warning_alerts'))
            )
        )
    ).withColumn(
        'maintenance_priority',
        F.when(F.col('remaining_days_method3') <= 30, 'High')
        .when(F.col('remaining_days_method3') <= 90, 'Medium')
        .otherwise('Low')
    )
    
    # Loại bỏ các cột tạm thời
    columns_to_drop = [f'deduction_{col}' for col in alert_columns]
    result_df = result_df.drop(*columns_to_drop)
    
    return result_df

# Kết quả DataFrame sẽ có thêm các cột mới:
# 1. current_score: Điểm số hiện tại (0-100)
# 2. num_danger_alerts: Số lượng cảnh báo Danger
# 3. num_warning_alerts: Số lượng cảnh báo Warning
# 4. health_status: Trạng thái sức khỏe (Critical/Warning/Healthy)
# 5. remaining_days: Số ngày còn lại dự kiến
# 6. maintenance_priority: Mức độ ưu tiên bảo trì (High/Medium/Low)
"""
# Giả sử thiết bị hoạt động bình thường có thể hoạt động 180 ngày (6 tháng)
# Mỗi cảnh báo Danger sẽ giảm 50% số ngày còn lại
# Mỗi cảnh báo Warning sẽ giảm 25% số ngày còn lại

# Ví dụ:
# - Thiết bị không có cảnh báo: 180 ngày
# - Thiết bị có 1 Danger: 180 * (1-0.5) = 90 ngày
# - Thiết bị có 1 Danger + 1 Warning: 180 * (1-0.5) * (1-0.25) = 67.5 ngày
"""

# Gọi hàm remaining_days_method3 với DataFrame df hiện tại
final_result3 = calculate_device_health(filtered_df)

# Hiển thị kết quả
final_result3.show(70)

+------------+--------------------+-----------+----------+----------+----------------+------------+-----------+---------+----------+-------------+----------+-------------+--------------------+---------------+------------+--------------------+-----------------------+---------------------+--------------------+---------+-----------+----------------+--------------------+-------------+--------------+---------+--------------+------------------+--------------------+---------------+------------------+------------------+-----------+------------+--------------------+-------------------+----------------+--------------------+----------------------+------------------+-------------------+------------------+-------------------+------------------+---------------------------+--------------------------+-------------+-------------------+------------------------+--------------------------+--------------------+----------------------------+----------------------+-----------------+--------------+------------

In [34]:
# in ra cột equipment_id, remaining_days_method1 của final_result1 nhưng chỉ in ra 1 giá trị trong mỗi nhóm equipment_id
final_result3.select('equipment_id', 'remaining_days_method3').distinct().show(1000)

+------------+----------------------+
|equipment_id|remaining_days_method3|
+------------+----------------------+
|          25|                 180.0|
|          41|                 180.0|
|          11|                   0.0|
|          30|                   0.0|
|           4|                 180.0|
|          28|                   0.0|
|          20|                 180.0|
|          29|                   0.0|
|          42|                 180.0|
|           1|                   0.0|
|          13|                 180.0|
|          18|                 180.0|
|           7|                   0.0|
|          33|                   0.0|
|          14|                 180.0|
|          19|                   0.0|
|          11|                 180.0|
|          40|                   0.0|
|           3|                 180.0|
|          35|                 180.0|
|          25|                   0.0|
|          37|                   0.0|
|          40|                 180.0|
|          3

# Weighted Average Method

In [45]:
# Gán trọng số cho từng phương pháp
weight_method1 = 0.8  # Phương pháp 1 
weight_method2 = 0.1  # Phương pháp 2
weight_method3 = 0.1  # Phương pháp 3 

# Gộp các DataFrame lại với nhau (join giữa final_result2 và final_result3 theo 'equipment_id')
final_result_combined = final_result2.join(
    final_result3.select('equipment_id', 'remaining_days_method3'),  # Chỉ lấy cột 'equipment_id' và 'remaining_days_method3'
    on='equipment_id',  # Cột để join là 'equipment_id'
    how='left'  # Phương thức join là 'left', giữ tất cả các dòng trong final_result2
)

# Lấy giá trị đầu tiên (hoặc bạn có thể thay bằng các hàm khác như F.max hoặc F.min) cho mỗi 'equipment_id'
final_result_combined = final_result_combined.groupBy('equipment_id').agg(
    F.first('remaining_days_method1').alias('remaining_days_method1'),
    F.first('remaining_days_method2').alias('remaining_days_method2'),
    F.first('remaining_days_method3').alias('remaining_days_method3')
)

# Tính toán số ngày còn lại tổng hợp theo phương pháp trung bình có trọng số và làm tròn
final_result_combined = final_result_combined.withColumn(
    'total_remaining_days',
    F.round(
        (F.col('remaining_days_method1') * weight_method1) + 
        (F.col('remaining_days_method2') * weight_method2) + 
        (F.col('remaining_days_method3') * weight_method3), 
        1  # Làm tròn đến 1 chữ số thập phân
    )
)

# Hiển thị kết quả
final_result_combined.show(70)

+------------+----------------------+----------------------+----------------------+--------------------+
|equipment_id|remaining_days_method1|remaining_days_method2|remaining_days_method3|total_remaining_days|
+------------+----------------------+----------------------+----------------------+--------------------+
|          12|                   9.4|                 195.7|                 180.0|                45.1|
|          22|                  15.3|                 338.2|                 180.0|                64.1|
|          26|                   2.8|                  60.2|                 180.0|                26.3|
|          27|                  17.9|                 370.5|                 180.0|                69.4|
|          28|                  13.6|                 300.7|                 180.0|                59.0|
|          31|                   2.7|                  59.2|                 180.0|                26.1|
|          34|                  15.6|                 3

In [46]:
# Lọc các thiết bị có equipment_id từ 1 đến 10 và in ra cột equipment_id và remaining_days_total
final_result_combined.filter(F.col('equipment_id').between(1, 10)) \
    .select('equipment_id', 'remaining_days_method1', 'remaining_days_method2', 'remaining_days_method3', 'total_remaining_days') \
    .dropDuplicates() \
    .show(1000)

+------------+----------------------+----------------------+----------------------+--------------------+
|equipment_id|remaining_days_method1|remaining_days_method2|remaining_days_method3|total_remaining_days|
+------------+----------------------+----------------------+----------------------+--------------------+
|           1|                   4.5|                  90.0|                 180.0|                30.6|
|           2|                  15.5|                 341.8|                   0.0|                46.6|
|           3|                   2.5|                  55.5|                 180.0|                25.6|
|           4|                  17.5|                 385.2|                   0.0|                52.5|
|           5|                   3.3|                  66.9|                   0.0|                 9.3|
|           6|                   3.8|                  85.4|                 180.0|                29.6|
|           7|                   2.3|                  

# timestamp_for_maintenance

In [47]:
from pyspark.sql import functions as F

# Thêm cột timestamp_for_maintenance
final_df = final_result_combined.withColumn(
    'timestamp_for_maintenance', 
    F.date_add(F.current_date(), F.col('total_remaining_days').cast('int'))  # Cộng số ngày vào ngày hiện tại
)

# Hiển thị kết quả
final_df.show(70)

+------------+----------------------+----------------------+----------------------+--------------------+-------------------------+
|equipment_id|remaining_days_method1|remaining_days_method2|remaining_days_method3|total_remaining_days|timestamp_for_maintenance|
+------------+----------------------+----------------------+----------------------+--------------------+-------------------------+
|          12|                   9.4|                 195.7|                 180.0|                45.1|               2025-01-05|
|          22|                  15.3|                 338.2|                 180.0|                64.1|               2025-01-24|
|          26|                   2.8|                  60.2|                 180.0|                26.3|               2024-12-17|
|          27|                  17.9|                 370.5|                 180.0|                69.4|               2025-01-29|
|          28|                  13.6|                 300.7|                 180.0|